In [28]:
from PIL import Image
import numpy as np
import os
import tensorflow as tf

IMAGE_SIZE = 128

cwd = os.getcwd()
dataset_dir = os.path.join(cwd,'dataset')

with open(os.path.join(dataset_dir,"artists.txt")) as file:
    artists = [artist for artist in file.readlines()][0].split(" ")

dirs = os.listdir(dataset_dir)
wanted_artists_dirs = [wanted_artist for wanted_artist in dirs if any(artist in wanted_artist.lower() for artist in artists)]
wanted_artists_dirs

from PIL import Image
import numpy as np

IMAGE_SIZE = 128
training_data = []
count = 0

for dir in wanted_artists_dirs:
    complete_dir = os.path.join(dataset_dir,dir)

    for filename in os.listdir(complete_dir):
        path = os.path.join(complete_dir, filename)
        image = Image.open(path).resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
        training_data.append(np.asarray(image))
        count += 1

print(f"Images:",count)       
training_data = np.reshape(training_data, (-1, IMAGE_SIZE, IMAGE_SIZE, 3))
training_data = training_data / 127.5 - 1

np.save('data.npy', training_data)

In [ ]:
# training_data = np.load("data.npy")

In [25]:
filenames = []

for dir in os.listdir(dataset_dir):
    subdir = os.path.join(dataset_dir,dir)
    for filename in os.listdir(subdir):
        filenames.append(filename)


In [29]:
from tensorflow.keras.applications.resnet50  import preprocess_input 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

class CustomDataset(tf.keras.utils.Sequence):

    def __init__(self, filenames, preprocessing_function=preprocess_input, out_shape=[IMAGE_SIZE,IMAGE_SIZE],img_generator=ImageDataGenerator(rescale=1./255 - 0.5)):

        self.filenames = filenames
        self.preprocessing_function = preprocessing_function
        self.out_shape = out_shape

    def __len__(self): return len(self.filenames)

    def __getitem__(self, index):

        # Read Image
        curr_filename = self.filenames[index]
        img = Image.open(join(training_dir, str(curr_filename) + '.png')).convert('RGB')

        # Resize image
        resized_img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
        img_arr = np.asarray(resized_img)

        return img_arr

In [30]:
train_custom_dataset = CustomDataset(filenames=filenames)

In [36]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_custom_dataset,
                                               output_types=(tf.uint8),
                                               output_shapes=([IMAGE_SIZE,IMAGE_SIZE, 3]))
train_dataset = train_dataset.batch(32)
train_dataset = train_dataset.repeat()
train_dataset

<RepeatDataset shapes: (None, 128, 128, 3), types: tf.uint8>

In [3]:
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam

# Preview image Frame
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 4
SAVE_FREQ = 100

# Size vector to generate images from
NOISE_SIZE = 100

# Configuration
EPOCHS = 10000 
BATCH_SIZE = 32
GENERATE_RES = 3
IMAGE_SIZE = 128 # rows/cols
IMAGE_CHANNELS = 3

In [4]:
def build_discriminator(image_shape):
   
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
   
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))
    
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    
    return Model(input_image, validity)

In [ ]:
def build_generator(noise_size, channels):
  
    model = Sequential()
    model.add(Dense(4 * 4 * 256, activation="relu", input_dim=noise_size))
    model.add(Reshape((4, 4, 256)))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    for i in range(GENERATE_RES):
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        
    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
    
    input = Input(shape=(noise_size,))
    generated_image = model(input)
    
    return Model(input, generated_image)

In [ ]:
def save_images(cnt, noise):
    
    rows = PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN))
    columns = PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN))

    image_array = np.full((rows,columns,3), 255, dtype=np.uint8)

    # predict a noise
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    
    image_count = 0
    
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c + IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1
            
    output_path = 'output'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)

#### Compile

In [5]:
optimizer = Adam(1.5e-4, 0.5)
loss = "binary_crossentropy"
metrics = ["accuracy"]

image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

## Discriminator

In [6]:
discriminator = build_discriminator(image_shape)

discriminator.compile(loss=loss, optimizer=optimizer, metrics=metrics)
discriminator.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 1)                 1720385   
Total params: 1,720,385
Trainable params: 1,718,465
Non-trainable params: 1,920
_________________________________________________________________


In [10]:
from tensorflow.keras.utils import plot_model

plot_model(discriminator, to_file='discr_plot.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


## Generator

In [ ]:
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)
generator.summary() 

## Model

In [ ]:
random_input = Input(shape=(NOISE_SIZE,))
generated_image = generator(random_input)
discriminator.trainable = False
validity = discriminator(generated_image)

combined = Model(random_input, validity)
combined.compile(loss=loss, optimizer=optimizer, metrics=metrics)
combined.summary()

## Training

In [ ]:
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))

fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE))
cnt = 1

for epoch in range(EPOCHS):
    idx = np.random.randint(0, training_data.shape[0], BATCH_SIZE)

    #taking a sample from a real image and putting that on x_real
    x_real = training_data[idx]

    # defining a noise vector
    noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE))
    # passing that to our generator model to generate a fake image in x_fake.
    x_fake = generator.predict(noise)

    # training our discriminator model in both real and fake images separately
    discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
    discriminator_metric_generated = discriminator.train_on_batch(x_fake, y_fake)
 
    discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)

    generator_metric = combined.train_on_batch(noise, y_real)

if epoch % 5 == 0:

    save_images(cnt, fixed_noise)
    cnt += 1
    print(f"{epoch} epoch, Discriminator accuracy: {100*  discriminator_metric[1]}, Generator accuracy: {100 * generator_metric[1]}")
    